In [50]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

In [51]:
%matplotlib inline

In [52]:
df = pd.read_csv('BoaData.csv')

In [53]:
pd.set_option('display.max_columns', None)

In [54]:
df.head()

,Timestamp,Jaką grupę krwi posiadasz?,"Jakie masz lub miałeś zdiagnozowane infekcje, lub mocno podejrzewasz, że je masz/miałeś ze względu na charakterystyczne objawy?",Miejsce zamieszkania (orientacyjnie),W którym miesiącu choroby zacząłeś się leczyć? Jeżeli nie zacząłeś - nie zaznaczaj,"Jaką metodę leczenia/terapii i jak długo stosujesz? Zaznacz każdą, którą stosujesz/stosowałeś. Jeżeli nie stosowałeś nie zaznaczaj. [standardowa kuracja - doxycyklina]","Jaką metodę leczenia/terapii i jak długo stosujesz? Zaznacz każdą, którą stosujesz/stosowałeś. Jeżeli nie stosowałeś nie zaznaczaj. [wiele antybiotyków skojarzonych ILADS]","Jaką metodę leczenia/terapii i jak długo stosujesz? Zaznacz każdą, którą stosujesz/stosowałeś. Jeżeli nie stosowałeś nie zaznaczaj. [protokół Buhnera]","Jaką metodę leczenia/terapii i jak długo stosujesz? Zaznacz każdą, którą stosujesz/stosowałeś. Jeżeli nie stosowałeś nie zaznaczaj. [protokół Cowdena]","Jaką metodę leczenia/terapii i jak długo stosujesz? Zaznacz każdą, którą stosujesz/stosowałeś. Jeżeli nie stosowałeś nie zaznaczaj. [zioła liposomalne]","Jaką metodę leczenia/terapii i jak długo stosujesz? Zaznacz każdą, którą stosujesz/stosowałeś. Jeżeli nie stosowałeś nie zaznaczaj. [inna kompozycja ziół]","Jaką metodę leczenia/terapii i jak długo stosujesz? Zaznacz każdą, którą stosujesz/stosowałeś. Jeżeli nie stosowałeś nie zaznaczaj. [duże dawki witaminy D]","Jaką metodę leczenia/terapii i jak długo stosujesz? Zaznacz każdą, którą stosujesz/stosowałeś. Jeżeli nie stosowałeś nie zaznaczaj. [suplementy i witaminy]","Jaką metodę leczenia/terapii i jak długo stosujesz? Zaznacz każdą, którą stosujesz/stosowałeś. Jeżeli nie stosowałeś nie zaznaczaj. [olejki eteryczne]","Jaką metodę leczenia/terapii i jak długo stosujesz? Zaznacz każdą, którą stosujesz/stosowałeś. Jeżeli nie stosowałeś nie zaznaczaj. [KAMBO]","Jaką metodę leczenia/terapii i jak długo stosujesz? Zaznacz każdą, którą stosujesz/stosowałeś. Jeżeli nie stosowałeś nie zaznaczaj. [generator plazmowy]","Jaką metodę leczenia/terapii i jak długo stosujesz? Zaznacz każdą, którą stosujesz/stosowałeś. Jeżeli nie stosowałeś nie zaznaczaj. [dieta bezcukrowa]","Jaką metodę leczenia/terapii i jak długo stosujesz? Zaznacz każdą, którą stosujesz/stosowałeś. Jeżeli nie stosowałeś nie zaznaczaj. [dieta bezglutenowa]","Jaką metodę leczenia/terapii i jak długo stosujesz? Zaznacz każdą, którą stosujesz/stosowałeś. Jeżeli nie stosowałeś nie zaznaczaj. [dieta bezmleczna]","Jaką metodę leczenia/terapii i jak długo stosujesz? Zaznacz każdą, którą stosujesz/stosowałeś. Jeżeli nie stosowałeś nie zaznaczaj. [biorezonans]","Jaką metodę leczenia/terapii i jak długo stosujesz? Zaznacz każdą, którą stosujesz/stosowałeś. Jeżeli nie stosowałeś nie zaznaczaj. [farmaceutyczne środki przeciwdrobnoustrojowe, takie jak przeciwmalaryczne, przeciwpierwotniacze lub środki przeciwpasożytnicze]","Jaką metodę leczenia/terapii i jak długo stosujesz? Zaznacz każdą, którą stosujesz/stosowałeś. Jeżeli nie stosowałeś nie zaznaczaj. [terapia tlenowa]","Jaką metodę leczenia/terapii i jak długo stosujesz? Zaznacz każdą, którą stosujesz/stosowałeś. Jeżeli nie stosowałeś nie zaznaczaj. [olej z nasion konopii]","Jaką metodę leczenia/terapii i jak długo stosujesz? Zaznacz każdą, którą stosujesz/stosowałeś. Jeżeli nie stosowałeś nie zaznaczaj. [Binauralne dźwięki]",Czy stosowałeś jakieś używki w trakcie leczenia? Jeżeli tak zaznacz jak często (orientacyjnie) [tytoń],Czy stosowałeś jakieś używki w trakcie leczenia? Jeżeli tak zaznacz jak często (orientacyjnie) [alkohol],Czy stosowałeś jakieś używki w trakcie leczenia? Jeżeli tak zaznacz jak często (orientacyjnie) [kawa],Czy stosowałeś jakieś używki w trakcie leczenia? Jeżeli tak zaznacz jak często (orientacyjnie) [marihuana],Czy stosowałeś jakieś używki w trakcie leczenia? Jeżeli tak zaznacz jak często (orientacyjnie) [inne],Ile różnych antybiotyków bierzesz/brałeś jednocześnie?,W jakiej formie przyjmowałeś antybiotyki?,Jakie objawy występują/występowa

In [55]:
len(df.columns)

66

In [56]:
df = df.drop(['Unnamed: 64', 'Unnamed: 65'], axis='columns')

In [57]:
colnames = 'record_time,blood,infection,residence,start_treat'.split(',')
treatments = 'doxy,ilads,buhner,cowden,liposomal,other_herbs,vitaminD,supp,oil,kambo,plasma,sugar-free,gluten-free,dairy-free,bioresonance,antimicrobial,oxygen,cannabis,binaural'.split(',')
colnames.extend(treatments)
stimulants = 'tobacco,alcohol,coffee,marijuana,other_stim'.split(',')
colnames.extend(stimulants)
colnames.extend(['num_antibiotics', 'method_antibiotics'])
symptoms = 'depression,unreal,irritability,sleep,conc,stupor,memory,stiffness,breath,fatigue,numb,strength,ache,arthralgia,headache,facial_muscle,abdominal,gastro_reflux,eye,ringing,light,sound,toothache,rash,hair,chest_pain,bladder,libido,weight,fever,Parkinson,muscle_decay'.split(',')
colnames.extend(symptoms)
colnames.append('effective')
df.columns = colnames

In [58]:
df.head()

,record_time,blood,infection,residence,start_treat,doxy,ilads,buhner,cowden,liposomal,other_herbs,vitaminD,supp,oil,kambo,plasma,sugar-free,gluten-free,dairy-free,bioresonance,antimicrobial,oxygen,cannabis,binaural,tobacco,alcohol,coffee,marijuana,other_stim,num_antibiotics,method_antibiotics,depression,unreal,irritability,sleep,conc,stupor,memory,stiffness,breath,fatigue,numb,strength,ache,arthralgia,headache,facial_muscle,abdominal,gastro_reflux,eye,ringing,light,sound,toothache,rash,hair,chest_pain,bladder,libido,weight,fever,Parkinson,muscle_decay,effective
0,2018/08/21 1:49:54 PM CET,0RH+ 32% Polaków,Borelioza;Bartonella;Pasożyty,Wieś,18 -24 miesiąc,1-3 m,4-7 m,1-3 m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,często (codziennie),NaN,często (codziennie),NaN,NaN,1-3,doustne ciągłe,Występują,NaN,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,1.0
1,2018/08/21 1:53:43 PM CET,B RH+ 15% Polaków,Borelioza;Bartonella;Babesia;Anaplazma;vir.Eps...,Małe miasto (do 300 tysięcy mieszkańców),6+ rok,NaN,4-5 lat,NaN,NaN,NaN,NaN,NaN,4-5 lat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-3,dożylnie mieszane,NaN,Występują,Występowały,Występowały,Występują,Występują,Występują,Występowały,Występowały,Występowały,Występują,Występują,Występowały,NaN,Występują,Występowały,Występowały,NaN,Występują,Występowały,Występowały,Występowały,Występowały,Występowały,NaN,Występowały,NaN,NaN,NaN,Występowały,Występowały,NaN,8.0
2,2018/08/21 1:54:25 PM CET,0RH+ 32% Polaków,Borelioza;Bartonella;Mykoplazma pneumoniae;Bab...,Wieś,8-12 miesiąc,NaN,13-18 m,NaN,NaN,NaN,NaN,13-18 m,13-18 m,NaN,NaN,NaN,13-18 m,13-18 m,NaN,13-18 m,13-18 m,NaN,13-18 m,NaN,NaN,NaN,często (codziennie),NaN,NaN,4-6,doustne ciągłe,Występowały,Występowały,Występowały,Występowały,Występowały,Występowały,Występowały,Występowały,Występowały,Występowały,Występowały,Występowały,Występowały,NaN,Występowały,NaN,Występowały,Występowały,Występowały,NaN,NaN,NaN,NaN,Występowały,Występowały,Występowały,NaN,Występowały,Występowały,Występowały,Występowały,Występowały,4.0
3,2018/08/21 1:54:43 PM CET,AB RH+ 7% Polaków,Borelioza;Bartonella;Babesia;Pasożyty;Toksyny,Wieś,3-6 rok,NaN,4-7 m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,często (codziennie),NaN,często (codziennie),NaN,NaN,1-3,doustne ciągłe,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,NaN,Występują,Występują,NaN,NaN,NaN,Występują,Występują,Występują,Występują,NaN,Występują,NaN,NaN,Występują,NaN,NaN,NaN,Występują,NaN,10.0
4,2018/08/21 1:58:15 PM CET,0RH+ 32% Polaków,Borelioza;Mykoplazma pneumoniae;Yersinia,Wieś,3-6 rok,NaN,1-3 m,NaN,NaN,NaN,NaN,1-3 m,NaN,NaN,NaN,NaN,1-3 m,1-3 m,1-3 m,NaN,NaN,NaN,NaN,1-3 m,NaN,NaN,NaN,NaN,NaN,1-3,doustne ciągłe,NaN,Występowały,NaN,Występowały,NaN,NaN,NaN,Występowały,NaN,Występowały,Występowały,NaN,Występowały,Występowały,Występowały,Występowały,Występowały,Występowały,Występowały,Występowały,Występowały,NaN,Występowały,Występowały,Występowały,Występowały,NaN,NaN,NaN,NaN,NaN,NaN,4.0


dropping empty or with 'effective- NaN' column and duplicated rows:

In [59]:
df = df.drop([21,22,28,30,50,68,105,106,110,122])

In [60]:
df.head(30)

,record_time,blood,infection,residence,start_treat,doxy,ilads,buhner,cowden,liposomal,other_herbs,vitaminD,supp,oil,kambo,plasma,sugar-free,gluten-free,dairy-free,bioresonance,antimicrobial,oxygen,cannabis,binaural,tobacco,alcohol,coffee,marijuana,other_stim,num_antibiotics,method_antibiotics,depression,unreal,irritability,sleep,conc,stupor,memory,stiffness,breath,fatigue,numb,strength,ache,arthralgia,headache,facial_muscle,abdominal,gastro_reflux,eye,ringing,light,sound,toothache,rash,hair,chest_pain,bladder,libido,weight,fever,Parkinson,muscle_decay,effective
0,2018/08/21 1:49:54 PM CET,0RH+ 32% Polaków,Borelioza;Bartonella;Pasożyty,Wieś,18 -24 miesiąc,1-3 m,4-7 m,1-3 m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,często (codziennie),NaN,często (codziennie),NaN,NaN,1-3,doustne ciągłe,Występują,NaN,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,1.0
1,2018/08/21 1:53:43 PM CET,B RH+ 15% Polaków,Borelioza;Bartonella;Babesia;Anaplazma;vir.Eps...,Małe miasto (do 300 tysięcy mieszkańców),6+ rok,NaN,4-5 lat,NaN,NaN,NaN,NaN,NaN,4-5 lat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-3,dożylnie mieszane,NaN,Występują,Występowały,Występowały,Występują,Występują,Występują,Występowały,Występowały,Występowały,Występują,Występują,Występowały,NaN,Występują,Występowały,Występowały,NaN,Występują,Występowały,Występowały,Występowały,Występowały,Występowały,NaN,Występowały,NaN,NaN,NaN,Występowały,Występowały,NaN,8.0
2,2018/08/21 1:54:25 PM CET,0RH+ 32% Polaków,Borelioza;Bartonella;Mykoplazma pneumoniae;Bab...,Wieś,8-12 miesiąc,NaN,13-18 m,NaN,NaN,NaN,NaN,13-18 m,13-18 m,NaN,NaN,NaN,13-18 m,13-18 m,NaN,13-18 m,13-18 m,NaN,13-18 m,NaN,NaN,NaN,często (codziennie),NaN,NaN,4-6,doustne ciągłe,Występowały,Występowały,Występowały,Występowały,Występowały,Występowały,Występowały,Występowały,Występowały,Występowały,Występowały,Występowały,Występowały,NaN,Występowały,NaN,Występowały,Występowały,Występowały,NaN,NaN,NaN,NaN,Występowały,Występowały,Występowały,NaN,Występowały,Występowały,Występowały,Występowały,Występowały,4.0
3,2018/08/21 1:54:43 PM CET,AB RH+ 7% Polaków,Borelioza;Bartonella;Babesia;Pasożyty;Toksyny,Wieś,3-6 rok,NaN,4-7 m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,często (codziennie),NaN,często (codziennie),NaN,NaN,1-3,doustne ciągłe,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,NaN,Występują,Występują,NaN,NaN,NaN,Występują,Występują,Występują,Występują,NaN,Występują,NaN,NaN,Występują,NaN,NaN,NaN,Występują,NaN,10.0
4,2018/08/21 1:58:15 PM CET,0RH+ 32% Polaków,Borelioza;Mykoplazma pneumoniae;Yersinia,Wieś,3-6 rok,NaN,1-3 m,NaN,NaN,NaN,NaN,1-3 m,NaN,NaN,NaN,NaN,1-3 m,1-3 m,1-3 m,NaN,NaN,NaN,NaN,1-3 m,NaN,NaN,NaN,NaN,NaN,1-3,doustne ciągłe,NaN,Występowały,NaN,Występowały,NaN,NaN,NaN,Występowały,NaN,Występowały,Występowały,NaN,Występowały,Występowały,Występowały,Występowały,Występowały,Występowały,Występowały,Występowały,Występowały,NaN,Występowały,Występowały,Występowały,Występowały,NaN,NaN,NaN,NaN,NaN,NaN,4.0
5,2018/08/21 2:04:10 PM CET,B RH+ 15% Polaków,Borelioza;Candidoza,Duże miasto (powyżej 300 tysięcy mieszkańców),1-3 miesiąc,1-3 m,NaN,NaN,NaN,NaN,NaN,1-3 m,1-3 m,1-3 m,NaN,NaN,NaN,NaN,1-3 m,NaN,1-3 m;4-7 m;8-12 m;13-18 m;19 -24 m;2-3 lat;4-...,NaN,NaN,1-3 m,NaN,NaN,średnio (1-2 razy w tygodniu),NaN,NaN,1-3,doustne pulsacyjnie,Występują,Występują,Występują,Występują,Występują,Występują,Występują,Występują,NaN,Występują,Występują,Występują,Występują,Występują,NaN,NaN,NaN,Występują,Występują,Występują,Występują,Występują,Występują,NaN,NaN,Występują,Występują,Występują,NaN,NaN,Występują,NaN,0.0
6,2018/08/21 2:05:35 PM CET,AB RH+ 7% Polaków,Borelioza;Bartonella;

In [61]:
df.iloc[:,:-1] = df.iloc[:,:-1].apply(lambda x: pd.factorize(x)[0])

In [62]:
df.head()

,record_time,blood,infection,residence,start_treat,doxy,ilads,buhner,cowden,liposomal,other_herbs,vitaminD,supp,oil,kambo,plasma,sugar-free,gluten-free,dairy-free,bioresonance,antimicrobial,oxygen,cannabis,binaural,tobacco,alcohol,coffee,marijuana,other_stim,num_antibiotics,method_antibiotics,depression,unreal,irritability,sleep,conc,stupor,memory,stiffness,breath,fatigue,numb,strength,ache,arthralgia,headache,facial_muscle,abdominal,gastro_reflux,eye,ringing,light,sound,toothache,rash,hair,chest_pain,bladder,libido,weight,fever,Parkinson,muscle_decay,effective
0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,-1,0,-1,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0
1,1,1,1,1,1,-1,1,-1,-1,-1,-1,-1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,1,-1,0,1,1,0,0,0,1,1,1,0,0,1,-1,0,1,1,-1,0,1,1,1,1,1,-1,1,-1,-1,-1,1,1,-1,8.0
2,2,0,2,0,2,-1,2,-1,-1,-1,-1,0,1,-1,-1,-1,0,0,-1,0,0,-1,0,-1,-1,-1,0,-1,-1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,1,-1,1,1,1,-1,-1,-1,-1,1,1,1,-1,1,1,1,1,1,4.0
3,3,2,3,0,3,-1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,-1,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,-1,-1,0,0,0,0,-1,0,-1,-1,0,-1,-1,-1,0,-1,10.0
4,4,0,4,0,3,-1,3,-1,-1,-1,-1,1,-1,-1,-1,-1,1,1,0,-1,-1,-1,-1,0,-1,-1,-1,-1,-1,0,0,-1,1,-1,1,-1,-1,-1,1,-1,1,1,-1,1,1,1,1,1,1,1,1,1,-1,1,1,1,1,-1,-1,-1,-1,-1,-1,4.0


In [63]:
X=df[['start_treat','doxy','ilads','buhner','cowden','liposomal','other_herbs','vitaminD','supp','oil','sugar-free','gluten-free','dairy-free','bioresonance','antimicrobial','oxygen','cannabis','binaural','tobacco','alcohol','coffee','marijuana','other_stim','num_antibiotics','method_antibiotics']].values
y=df['effective'].values

In [24]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [25]:
X = sc.fit_transform(X)

C:\Users\Dzejkot\Anaconda3\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


In [68]:
X

array([[ 0,  0,  0, ..., -1,  0,  0],
       [ 1, -1,  1, ..., -1,  0,  1],
       [ 2, -1,  2, ..., -1,  1,  0],
       ...,
       [ 1, -1,  2, ..., -1,  1,  0],
       [-1, -1,  3, ..., -1,  0,  4],
       [ 6, -1,  3, ..., -1,  0,  8]], dtype=int64)

In [69]:
model = DecisionTreeRegressor(max_depth=10)
cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')

array([-3.16489362, -2.68432007, -2.93478261])

In [70]:
model = RandomForestRegressor(max_depth=10, n_estimators=25, n_jobs=8)
scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
np.mean(scores), np.std(scores)

(-2.4904417062744364, 0.20968351147042355)